In [ ]:
from __future__ import annotations
from pprint import pprint

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

## 1

### Загрузка, предобработка, визуализация

> 1. Выбор датасета:    
>    - Датасет о пассажирах Титаника: [Titanic Dataset](https://www.kaggle.com/c/titanic)
>    - Датасет о диабете: [Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database)
>    - Загрузите выбранный датасет и выполните предварительную обработку данных.
>    - Получите и визуализируйте (графически) статистику по датасету (включая
>      количество, среднее значение, стандартное отклонение, минимум, максимум и
>      различные квантили).


In [ ]:
ds = pd.read_csv("/content/diabetes.csv")
ds.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
248,9,124,70,33,402,35.4,0.282,34,0
753,0,181,88,44,510,43.3,0.222,26,1
31,3,158,76,36,245,31.6,0.851,28,1
62,5,44,62,0,0,25.0,0.587,36,0
241,4,91,70,32,88,33.1,0.446,22,0


In [ ]:
ds.info(), ds.describe(), f'is there any nulls? { ds.isna().any().any() }'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


(None,
        Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
 count   768.000000  768.000000     768.000000     768.000000  768.000000   
 mean      3.845052  120.894531      69.105469      20.536458   79.799479   
 std       3.369578   31.972618      19.355807      15.952218  115.244002   
 min       0.000000    0.000000       0.000000       0.000000    0.000000   
 25%       1.000000   99.000000      62.000000       0.000000    0.000000   
 50%       3.000000  117.000000      72.000000      23.000000   30.500000   
 75%       6.000000  140.250000      80.000000      32.000000  127.250000   
 max      17.000000  199.000000     122.000000      99.000000  846.000000   
 
               BMI  DiabetesPedigreeFunction         Age     Outcome  
 count  768.000000                768.000000  768.000000  768.000000  
 mean    31.992578                  0.471876   33.240885    0.348958  
 std      7.884160                  0.331329   11.760232    0.476951  
 min      0.00

Типы распознаны, данные без налов, но есть некорректные значения, например, 'BloodPressure'=0. Их стоит заменить другими значениями. Возьмем для этого среднее.

In [ ]:
cols_fix = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
ds[cols_fix] = ds.replace(0, ds.mean(axis=0))[cols_fix]
ds.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.681605,72.254807,26.606479,118.660163,32.450805,0.471876,33.240885,0.348958
std,3.369578,30.436016,12.115932,9.631241,93.080358,6.875374,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,20.536458,79.799479,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,79.799479,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
sns.pairplot(ds, hue='Outcome')

Смотря на гистограммы можно сказать, что датасет, в принципе, сбалансированный: распределения нормальные +-, есть пики в некоторых признаках, `Outcome` соотносится $\approx 2:1$ (с этим, возможно, нужно будет что-то делать). Видно, что класы визуально плохо отделяются, хотя и есть некоторая *грань*.

### Разделение на обуч./тест.

> - Разделите данные на обучающий и тестовый наборы в
>   соотношении, которое вы считаете подходящим.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ds.drop('Outcome', axis=1), ds['Outcome'], test_size=0.3)

### Модель Логистической регрессии

> - Реализуйте логистическую регрессию "с нуля" без использования
>   сторонних библиотек, кроме NumPy и Pandas.
>   Ваша реализация логистической регрессии должна включать в себя:
>    - Функцию для вычисления гипотезы (sigmoid function).
>    - Функцию для вычисления функции потерь (log loss).
>    - Метод обучения, который включает в себя градиентный спуск.
>    - Возможность варьировать гиперпараметры, такие как
>      коэффициент обучения (learning rate) и количество итераций.


**Предсказания:**
$$
y_{pred}(x, w) = \frac{1}{1 + e^{-\langle x, w \rangle}}
$$

**Лосс (LogLoss):**
$$
L(w) = -y\, log\,y_{pred} - (1-y)\,log\,(1-y_{pred})
$$

**Градиент:**
$$
\frac{\partial{L}}{\partial{w}}
= \left(-\frac{y}{y_{pred}} + \frac{1-y}{1-y_{pred}}\right)\frac{\partial{y_{pred}}}{\partial{w}}
$$

$$
\frac{\partial{y_{pred}}}{\partial{w}} = \frac{-1}{(1+e^{-\langle x, w \rangle})^2} e^{-\langle x, w \rangle} (-x) = y_{pred}(1-y_{pred})x
$$

$$
\frac{\partial{L}}{\partial{w}} = (y_{pred} - y) x
$$

In [ ]:
def logit(x, w):
    print("logit")
    print(np.dot(x, w))
    return np.dot(x, w)

def sigmoid(h):
    return 1.0 / (1 + np.exp(-h))

class LogisticRegression(object):
    def __init__(self):
        self.w = None

    def fit(self, X, y, max_iter: int=100, lr: float=0.1):
        n, k = X.shape

        if self.w is None:
            self.w = np.random.randn(k + 1)

        X_train = np.concatenate((np.ones((n, 1)), X), axis=1)

        losses = []

        for iter_num in range(max_iter):
            z = sigmoid(logit(X_train, self.w))
            grad = ((X_train.T) @ (z - y)) / len(y)

            self.w -= grad * lr

            losses.append(self.__loss(y, z))

        return losses

    def predict_proba(self, X):
        n, k = X.shape
        X_ = np.concatenate((np.ones((n, 1)), X), axis=1)
        return sigmoid(logit(X_, self.w))

    def predict(self, X, threshold: float=0.5):
        return self.predict_proba(X) >= threshold

    def get_weights(self):
        return self.w

    def __loss(self, y, p):
        p = np.clip(p, 1e-10, 1 - 1e-10)
        return np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

## 2-3 Тестирование

> 2. Исследование гиперпараметров:
>   - Проведите исследование влияния гиперпараметров на
>     производительность модели. Варьируйте следующие гиперпараметры:
>   - Коэффициент обучения (learning rate).
>   - Количество итераций обучения.
>   - Метод оптимизации (например, градиентный спуск или оптимизация Ньютона).


> 3. Оценка модели:
>    - Для каждой комбинации гиперпараметров оцените
>      производительность модели на тестовом наборе данных, используя метрики,
>      такие как accuracy, precision, recall и F1-Score.


Пункты 2 и 3 сделаем сразу, тем более не требуется самим реализовать метрики:

In [ ]:
sc = MinMaxScaler().fit(X_train)
best_model = {
    'f1': -1,
    'accuracy': -1,
    'precision': -1,
    'recall': -1,
    'learning_rate': -1,
    'max_iterations': -1,
    'model': None,
}


for learning_rate in [0.01, 0.05, 0.1, 0.2, 0.5, 0.7]:
    for max_iterations in [100, 200, 500, 1000, 1500, 2000, 3000]:

        model = LogisticRegression()
        model.fit(sc.transform(X_train), y_train, max_iter=max_iterations, lr=learning_rate)
        y_pred = model.predict(sc.transform(X_test))

        _accuracy = accuracy_score(y_test, y_pred)
        _precision = precision_score(y_test, y_pred)
        _recall = recall_score(y_test, y_pred)
        _f1 = f1_score(y_test, y_pred)

        if False: # verbose?
            print(f"{learning_rate=}, {max_iterations=}")
            print(f"Accuracy:   {_accuracy}")
            print(f"Precision:  {_precision}")
            print(f"Recall:     {_recall}")
            print(f"F1:         {_f1}")

        if _f1 > best_model['f1']:
            best_model['model'] = model
            best_model['f1'] = _f1
            best_model['accuracy'] = _accuracy
            best_model['precision'] = _precision
            best_model['recall'] = _recall
            best_model['learning_rate'] = learning_rate
            best_model['max_iterations'] = max_iterations

print(f"Best model:")
pprint(best_model)

Streaming output truncated to the last 5000 lines.
 -5.49932193e-01 -6.74218397e-01  1.86590902e-01 -1.62734403e-01
  3.75139220e-01 -3.48547225e-01 -7.22081465e-01 -4.33009913e-01
 -5.70729463e-01  3.68716387e-01 -7.89029029e-01 -3.52595283e-01
 -5.93779586e-02  1.49255565e-01 -5.27836588e-01  8.10076342e-01
 -4.57949609e-01 -1.02833274e+00  7.64438417e-02 -3.19622962e-01
 -7.71164780e-01 -3.84231316e-01  1.66938351e-02 -5.52725613e-01
 -1.66683780e-01 -1.08213400e-01 -1.41651797e-01 -5.83434067e-01
  1.09279989e-01 -1.55594691e-01  3.80775815e-01 -2.98846351e-01
 -3.63030572e-01 -7.74456682e-02 -9.36795125e-03  2.00625641e-01
 -4.83095729e-01 -5.39817985e-02  5.30244765e-01 -5.03773565e-01
 -2.49743725e-01 -3.98924728e-01 -5.04245845e-01 -2.84221754e-01
 -8.49892711e-02 -6.43173944e-01  5.58092636e-01  2.42546946e-01
 -2.69719499e-01 -1.46079811e+00 -6.66136740e-01 -5.27380562e-01
  5.29663810e-01 -5.72920318e-01  7.13235530e-02 -6.03837671e-01
 -1.94993942e-01  4.25592614e-01 -6.240

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
 -3.84984638e-01 -6.23982831e-01 -7.40099161e-01 -2.14850746e-01
 -3.00964178e-01 -8.81523139e-02 -3.12835626e-01 -1.77271691e-01
 -2.12848708e-01  2.00650205e-02 -6.57928627e-02 -2.20647117e-01
 -2.07755742e-01 -1.95669215e-02  7.73700942e-02 -2.57482501e-01
 -2.56199838e-01 -2.48823944e-02 -7.40586548e-01  2.84501999e-02
  6.15552845e-03 -1.41338869e-01 -2.90726474e-01  8.35502848e-02
 -3.25075491e-01 -4.06386876e-01 -6.13724343e-02 -1.22343563e-01
  8.90087725e-02 -3.96393956e-01 -4.22983476e-01 -2.14018418e-02
 -2.87432757e-01 -3.07573260e-01  1.39203172e-02 -1.84068571e-01
 -5.21329002e-01 -3.99489887e-01 -3.03690038e-01  1.81137600e-02
 -4.73502620e-01 -6.24130364e-01 -7.53702354e-01  2.10483202e-01
 -2.52473541e-01]
logit
[-2.24310576e-01 -7.15966126e-01  9.74672386e-02 -2.00118662e-01
  1.11152678e-01 -2.35764477e-01  3.32235228e-01 -3.39897364e-02
 -4.13438507e-02 -8.13440520e-01  2.24512200e-01  2.74157953e-01
 -2.65793290e-0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
  2.74840965e-01 -7.60558324e-01 -3.96084524e-01 -4.30926216e-01
 -5.31326370e-01 -4.80907922e-01 -2.54744017e-01 -4.71921893e-01
 -4.15889175e-01 -4.39068298e-01 -5.86768200e-01 -2.80221133e-01
 -9.12568390e-02 -5.21488769e-01 -1.70502481e-01 -2.40767421e-01
 -7.04406431e-01 -5.66768367e-01 -4.57112969e-01 -4.62947763e-01
 -2.75823180e-01 -5.13903477e-01 -3.99411682e-01 -3.86914507e-01
 -3.96995556e-01 -7.74975359e-01 -7.28133994e-01 -1.04413911e-02
 -5.00080956e-01 -2.33353478e-02 -5.74562912e-02 -2.71775283e-01
 -2.55109478e-01 -7.61695774e-01 -7.34888738e-01 -1.31674173e-01
 -5.39792211e-01 -4.43503244e-01 -1.35121592e-01 -5.46687339e-01
 -3.99217415e-01 -4.58533527e-02 -6.71486210e-01 -1.00178558e+00
 -3.04625359e-01  1.51061662e-03 -2.61547297e-01 -4.72518443e-01
 -5.03244885e-01 -6.34140549e-01 -8.75667674e-01 -5.67166436e-02
 -9.95159647e-02 -3.05778197e-01 -7.04426490e-01 -4.06752195e-02
 -1.79415693e-01 -6.27931940e-01 -1.620

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
 -9.86409892e-01 -4.48171256e-01 -9.11650387e-01 -7.46954132e-01
 -7.08345339e-01 -7.61547421e-01 -7.27161391e-01 -7.12580293e-01
  1.16725750e-02 -8.23498192e-01 -4.54971309e-01 -3.56638055e-01
 -7.29607154e-01  5.64034018e-01 -1.01163557e+00 -5.59875858e-01
 -8.18061838e-01 -7.46001395e-01 -1.20854555e+00 -5.40921654e-01
 -2.92514041e-01 -3.24633970e-01 -9.86960715e-01 -3.60631334e-01
 -1.35802636e-01  7.42248980e-01 -7.52932781e-01 -4.01825347e-01
 -5.21863755e-01  1.43062897e-01 -5.16715647e-01  1.19596465e-01
 -3.76558344e-01 -1.01370336e+00 -6.70340481e-01 -1.11871789e+00
 -7.97265651e-01 -5.42808166e-01 -6.88664284e-01 -5.19761410e-01
 -3.72012171e-01 -3.88639221e-01 -5.02625529e-01  2.34630975e-01
 -6.92299848e-01  3.92341311e-01 -4.74378479e-01 -6.50464781e-01
 -4.73533166e-01 -3.05855017e-01 -1.11566557e+00 -8.68158008e-01
 -9.51930190e-01 -1.01270094e+00 -7.47219039e-01 -5.17433899e-01
  2.43984093e-02 -8.36729947e-01 -1.237

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
 -3.16414747e-01 -4.47674405e-01 -9.42382247e-01 -1.04743867e-01
 -2.96230429e-01 -7.89734962e-01 -2.00339484e-01 -3.44223407e-01
 -4.03469471e-01 -1.98521571e-01 -6.97412286e-01 -5.64416676e-01
 -4.44336168e-01 -2.57545773e-01 -5.10015578e-01 -4.61485541e-01
 -5.34286464e-01 -9.47402556e-01 -4.60108018e-01 -5.94307277e-01
 -7.55848502e-01 -8.68105898e-01 -6.80878138e-02 -5.70484657e-02
 -2.72552591e-01 -3.47724939e-01 -5.97973961e-01 -5.98892559e-01
 -7.11168024e-01 -7.15784300e-01 -4.30621322e-01 -1.92578832e-01
 -4.89288567e-01 -8.66016605e-01 -4.18452230e-01 -5.89242828e-01
 -8.21342255e-01 -6.71559774e-01 -1.00649450e+00 -1.48491630e-01
 -4.19487600e-01 -5.59163577e-01 -6.90009367e-01 -7.31194187e-01
 -1.12362212e+00  1.31584097e-02 -1.24267480e-01  6.58468303e-02
 -1.12508353e+00 -3.90032691e-01 -2.41269979e-01 -4.92554754e-01
 -3.28953149e-01 -2.07735767e-01 -6.26275681e-01 -6.56403666e-01
 -8.41951432e-01 -4.64658813e-01 -1.008

KeyboardInterrupt: ignored

## Выводы

> Сделайте выводы о том, какие значения гиперпараметров наилучшим образом
работают для данного набора данных и задачи классификации. Обратите внимание
на изменение производительности модели при варьировании гиперпараметров.

Самописная модель логистической регрессии оказалась довольно производительной. Наилучшее качество пердстказания проявляется при `learning_rate=0.5`, `max_iter=3000`. Видимо количество итераций можно увеличивать безгранично — просто качество не будет расти.